<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/FacialCartoonization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# githubのコードを取得
!git clone https://github.com/SystemErrorWang/FacialCartoonization.git
%cd FacialCartoonization/

In [ ]:
# フォルダー内表示関数定義
import matplotlib.pyplot as plt
from PIL import Image
import os
import shutil
import numpy as np
%matplotlib inline

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        if file=='.ipynb_checkpoints':
           continue
        if file=='.DS_Store':
           continue
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(file, fontsize=15)               
    plt.show()
    plt.close()

# 静止画をアニメに変換


In [ ]:
# サンプル画像表示
display_pic('images')

In [ ]:
# result フォルダーリセット
if os.path.isdir('results'):
    shutil.rmtree('results')

# アニメ化の実行
!python inference.py

# 結果表示
display_pic('results')

・results フォルダー内のファイル名を00000.jpgからの連番にし、動画に変換します

In [ ]:
# ファイル名を00000.jpgからの連番に変更
import glob
files = glob.glob('./results/*.jpg')
files.sort()

for i, file in enumerate(files):
    img = Image.open(file)
    img.save('./results/'+str(i).zfill(5)+'.jpg')
    os.remove(file)

# 実写＋アニメ顔をmp4に変換
!ffmpeg -r 1 -i results/%05d.jpg -vcodec libx264 -pix_fmt yuv420p out.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./out.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# mp4動画をアニメに変換
・カレントディレクトリにmp4動画を保存してから実行して下さい。\
・動画は正方形の形状にして下さい。\
・11行目のビデオ指定を保存したファイル名に変更して下さい。

In [ ]:
# ビデオを静止画に変換
import os
import shutil
import cv2

# imagesフォルダーリセット
if os.path.isdir('images'):
    shutil.rmtree('images')
os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './sample.mp4',   # ビデオ指定
                   image_dir='./images/', 
                   image_file='%s.jpg'):  
 
    # Initial setting
    i = 0
    interval = 3
    length = 600  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:
           frame = cv2.resize(frame, (256, 256))  # 256×256にリサイズ  
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
# resultsフォルダーリセット
if os.path.isdir('results'):
    shutil.rmtree('results')

# アニメ化の実行    
!python inference.py

# output.mp4をリセット
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

# アニメ顔をmp4に変換
!ffmpeg -r 10 -i results/%06d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")